In [ ]:
import sys, os
import json
import datetime
import torch
import pandas as pd

import argparse

from tqdm import tqdm

from multi_turn_test import llm_pipeline
from multi_turn_test import MultiTurnConversation as mtc


choose_lm = 4 #gemma2_9B

# make dialogue wrapper
all_lms = ["qwen2_0B5", "qwen2_1B5", "qwen2_7B", "llama3_8B", "gemma2_9B", "mistral03_7B"]
test_lm = all_lms[choose_lm]

LLM_DS = {
    "gemma2_9B": llm_pipeline.gemma_2_9B_pipeline(),
}
obj_llm = LLM_DS[test_lm]
pipe, pipe_config, = obj_llm.pipe_init()

# make path
result_folder = "result/MTMT/"+test_lm
os.makedirs(result_folder,exist_ok=True)
result_path = result_folder+"/{}.{}"

# start running
MRs = [
    "P0_round_original",
    "P1_round_shuffle",
    "P2_round_reduce",
    "P3_round_duplicate",
    "P4_round_reduce_shuffle",
    "P5_round_shuffle_duplicate",
    "MR10_SNP_s06_synonym_replacement",
    "MR11_SNP_s06_introduce_typos",
    "MR12_SNP_s06_to_leet",
    "MR13_SNP_s06_add_words",
    ]

for dataset_name in MRs:
    print(test_lm, dataset_name)
    path_dataset = "data/{}.json".format(dataset_name)

    def current_time_string():
        return datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

    print(current_time_string(),"Start!")

    df_result = pd.DataFrame()

    dataset_MTMT = json.load(open(path_dataset))
    dataset_dev_full= json.load(open("data/extracted_dev_all_final_update_3.json"))
    all_keys = list(dataset_MTMT.keys())[:]
    template_system_info = "You are a smart reader. Please read the following story:\n{}\nI'm going to ask you some questions about the story. Please give me concise, exact, and short answer. If you are not sure about what I am asking, or any information in question is ambiguous, answer me 'unknown'. \nHere comes the first question: {}"

    for this_dialogue_key in tqdm(all_keys):
        this_dialogue = dataset_MTMT[this_dialogue_key]
        this_dialogue_round_keys = list(this_dialogue.keys())

        story_id = this_dialogue_key
        story_content = dataset_dev_full[story_id]["story_material"]

        # start conversation!
        this_conv = mtc.multi_turn_conversation(f_llm_pipeline=pipe, pipeline_params=pipe_config)
       
        for round_i, round_key in enumerate(this_dialogue_round_keys):

            thisround_question = this_dialogue[round_key]["Question"]
            thisround_goldanswer = this_dialogue[round_key]["Answer"]
            round_note = this_dialogue[round_key]
            if round_i == 0: 
                this_conv.new_rounds(message_body=template_system_info.format(story_content, thisround_question), expected_answer=[thisround_goldanswer], source_uuid=story_id, note=round_note)
            else:
                this_conv.new_rounds(message_body=thisround_question, expected_answer=[thisround_goldanswer], source_uuid=story_id, note=round_note)
                
        df_result=pd.concat([df_result,this_conv.end_chat(evaluate=True)]).reset_index(drop=True)

    df_result.to_pickle(result_path.format(dataset_name,"pickle"))
    df_result.to_csv(result_path.format(dataset_name,"csv"))
    print(current_time_string(),"End")

del pipe, pipe_config, obj_llm
torch.cuda.empty_cache()

In [ ]:
import sys, os
import json
import datetime
import torch
import pandas as pd

import argparse

from tqdm import tqdm

from multi_turn_test import llm_pipeline
from multi_turn_test import MultiTurnConversation as mtc

# make dialogue wrapper
all_lms = ["qwen2_0B5", "qwen2_1B5", "qwen2_7B", "llama3_8B", "mistral03_7B"]

MRs = [
    "P0_round_original",
    "P1_round_shuffle",
    "P2_round_reduce",
    "P3_round_duplicate",
    "P4_round_reduce_shuffle",
    "P5_round_shuffle_duplicate",
    "MR10_SNP_s06_synonym_replacement",
    "MR11_SNP_s06_introduce_typos",
    "MR12_SNP_s06_to_leet",
    "MR13_SNP_s06_add_words",
    ]

for choose_lm in range(len(all_lms)):
    test_lm = all_lms[choose_lm]

    LLM_DS = {
        "qwen2_0B5": llm_pipeline.qwen_2_0B5_pipeline(),
        "qwen2_1B5": llm_pipeline.qwen_2_1B5_pipeline(),
        "qwen2_7B": llm_pipeline.qwen_2_7B_pipeline(),
        "llama3_8B": llm_pipeline.llama_3_8B_pipeline(),
        "gemma2_9B": llm_pipeline.gemma_2_9B_pipeline(),
        "mistral03_7B": llm_pipeline.mistral03_7B_pipeline(),
    }
    obj_llm = LLM_DS[test_lm]
    pipe, pipe_config, = obj_llm.pipe_init()

    # make path
    result_folder = "result/MTMT/"+test_lm
    os.makedirs(result_folder,exist_ok=True)
    result_path = result_folder+"/{}.{}"

    for dataset_name in MRs:
        print(test_lm, dataset_name)
        path_dataset = "data/{}.json".format(dataset_name)

        def current_time_string():
            return datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

        print(current_time_string(),"Start!")

        df_result = pd.DataFrame()

        dataset_MTMT = json.load(open(path_dataset))
        dataset_dev_full= json.load(open("data/extracted_dev_all_final_update_3.json"))
        all_keys = list(dataset_MTMT.keys())[:]
        template_system_info = "You are a smart reader. Please read the following story:\n{}\nI'm going to ask you some questions about the story. Please give me concise, exact, and short answer. If you are not sure about what I am asking, or any information in question is ambiguous, answer me 'unknown'."

        for this_dialogue_key in tqdm(all_keys):
            this_dialogue = dataset_MTMT[this_dialogue_key]
            this_dialogue_round_keys = list(this_dialogue.keys())

            story_id = this_dialogue_key
            story_content = dataset_dev_full[story_id]["story_material"]

            # start conversation!
            this_conv = mtc.multi_turn_conversation(f_llm_pipeline=pipe, pipeline_params=pipe_config)
            this_conv.new_rounds(message_body=template_system_info.format(story_content), role="system", source_uuid=story_id, send_to_lm=False)

            for round_key in this_dialogue_round_keys:
                thisround_question = this_dialogue[round_key]["Question"]
                thisround_goldanswer = this_dialogue[round_key]["Answer"]
                round_note = this_dialogue[round_key]
                returned_msg = this_conv.new_rounds(message_body=thisround_question, expected_answer=[thisround_goldanswer], source_uuid=story_id, note=round_note)
                # print("Q:{}\nExpect A:{}\nLLM A:{}\n".format(thisround_question, thisround_goldanswer, returned_msg))

            df_result=pd.concat([df_result,this_conv.end_chat(evaluate=True)]).reset_index(drop=True)

        df_result.to_pickle(result_path.format(dataset_name,"pickle"))
        df_result.to_csv(result_path.format(dataset_name,"csv"))
        print(current_time_string(),"End")

    del pipe, pipe_config, obj_llm
    torch.cuda.empty_cache()